In [2]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
from requests.packages.urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
import json
import pandas as pd
import numpy as np
import arrow
import sys

# Data Scraping

## Get the list of all coins on CoinGecko

In [3]:
# Start a session with Requests
session = Session()

# Get the list of all coins ID (also scraped on CoinGecko)
url = 'https://api.coingecko.com/api/v3/coins/list'
coins_list = session.get(url).json()

In [ ]:
# Save if necessary
#with open('coins_list.json', 'w', encoding='utf-8') as f:
#    json.dump(coins_list, f, ensure_ascii=False, indent=4)

## Get all coins data

In [48]:
# Define a rety strategy, so that the website does not return erros
retry_strategy = Retry(
    total=5,
    status_forcelist=[429, 500, 502, 503, 504],
    method_whitelist=["HEAD", "GET", "OPTIONS"],
    backoff_factor=2
)
adapter = HTTPAdapter(max_retries=retry_strategy)
session.mount("https://", adapter)
session.mount("http://", adapter)

def get_coin_data(coin_id):
    """Given a coin id (from CoinGecko) retrieve the corresponding
    information."""
    
    url = 'https://api.coingecko.com/api/v3/coins/' + coin_id
    
    try:
        data = session.get(url).json()
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        print(e)
    
    return data

In [ ]:
# Load the list of all coins ID (also scraped on CoinGecko)
#with open('coins_list.json') as infile: 
#    coins_list = json.load(infile)

In [56]:
# Scrape data for all coins using the list
all_data = []
for ind, coin in enumerate(coins_list):
    sys.stdout.write('\r'+str(ind+1) + '/' + str(len(list_coins)))
    all_data.append(get_coin_data(coin['id']))

6251/6251

In [57]:
# Save all the data
#with open('all_coins_data.json', 'w', encoding='utf-8') as f:
#    json.dump(all_data, f, ensure_ascii=False, indent=4)

## Process and clean the data


In [2]:
# Load all coins data
#with open('all_coins_data.json') as infile: 
#    all_coins_data = json.load(infile)

In [6]:
# List all columns we wish to keep (see Coingecko API for columns' names)
columns_simple = [
    'id',
    'name',
    'symbol',
    'country_origin',
    'genesis_date',
    'sentiment_votes_up_percentage',
    'sentiment_votes_down_percentage',
    'market_cap_rank',
    'coingecko_rank',
    'coingecko_score',
    'developer_score',
    'community_score',
    'liquidity_score',
    'public_interest_score'
]

columns_market_data = [
    'price_change_percentage_24h',
    'price_change_percentage_7d',
    'price_change_percentage_14d',
    'price_change_percentage_30d',
    'price_change_percentage_60d',
    'price_change_percentage_200d',
    'price_change_percentage_1y',
    'total_supply',
    'max_supply',
    'circulating_supply'
]

columns_market_data_usd = [
    'ath_date',
    'atl_date',
    'ath', # USD
    'ath_change_percentage', # USD
    'atl', # USD
    'atl_change_percentage', # USD
    'market_cap', # USD
    'total_volume', # USD
]

columns_developer_data = [
    'stars',
    'subscribers',
    'total_issues',
    'closed_issues',
    'pull_requests_merged',
    'pull_request_contributors',
]

In [7]:
def clean_coin_json(coin_json):
    """Given the JSON (dict) with data about a coin, returns a new
    JSON (dict) object with the desired data"""
    
    cleaned_json = dict()
    for col in columns_simple:
        try:
            cleaned_json[col] = coin_json[col]
        except KeyError:
            cleaned_json[col] = np.nan
            print('Could not find key:', col)
            
    
    for col in columns_market_data:
        try:
            cleaned_json[col] = coin_json['market_data'][col]
        except KeyError:
            cleaned_json[col] = np.nan
            print('Could not find key:', col)
        
    for col in columns_market_data_usd:
        try:
            cleaned_json[col] = coin_json['market_data'][col]['usd']
        except KeyError:
            cleaned_json[col] = np.nan
            print('Could not find key:', col)
        
    cleaned_json.update(coin_json['community_data'])
    
    for col in columns_developer_data:
        try:
            cleaned_json['git_' + col] = coin_json['developer_data'][col]
        except KeyError:
            cleaned_json[col] = np.nan
            print('Could not find key:', col)
        
    cleaned_json.update(coin_json['public_interest_stats'])
    
    list_platforms_with_pairs = [{'platform': pair['market']['name'], 'pair': pair['base'] + '/' + pair['target']} for pair in coin_json['tickers']]
        
    cleaned_json['platforms_pairs'] = list_platforms_with_pairs
        
    return cleaned_json

In [48]:
# Clean the data
all_data_cleaned = [clean_coin_json(coin_data) for i, coin_data in enumerate(all_coins_data)]

Could not find key: ath_date
Could not find key: atl_date
Could not find key: ath
Could not find key: atl
Could not find key: market_cap
Could not find key: total_volume
Could not find key: ath_date
Could not find key: atl_date
Could not find key: ath
Could not find key: atl
Could not find key: market_cap
Could not find key: total_volume
Could not find key: ath_date
Could not find key: atl_date
Could not find key: ath
Could not find key: atl
Could not find key: market_cap
Could not find key: total_volume
Could not find key: ath_date
Could not find key: atl_date
Could not find key: ath
Could not find key: atl
Could not find key: market_cap
Could not find key: total_volume
Could not find key: ath_date
Could not find key: atl_date
Could not find key: ath
Could not find key: atl
Could not find key: market_cap
Could not find key: total_volume
Could not find key: ath_date
Could not find key: atl_date
Could not find key: ath
Could not find key: atl
Could not find key: market_cap
Could not fin

In [50]:
# Save as CSV
pd.DataFrame(all_data_cleaned).to_csv('coins_data.csv', index=False)

## Exploration

In [60]:
coins_df = pd.read_csv('coins_data.csv')
coins_df = coins_df[coins_df['atl_date'].notnull()]
coins_df['atl_date'] = coins_df['atl_date'].apply(lambda date: arrow.get(date))
coins_df['growth_ratio'] = coins_df['ath'] / coins_df['atl']

In [61]:
a_year_ago = arrow.utcnow().shift(months=-5)
coins_df[coins_df['atl_date']>= a_year_ago]

,id,name,symbol,country_origin,genesis_date,sentiment_votes_up_percentage,sentiment_votes_down_percentage,market_cap_rank,coingecko_rank,coingecko_score,...,git_stars,git_subscribers,git_total_issues,git_closed_issues,git_pull_requests_merged,git_pull_request_contributors,alexa_rank,bing_matches,platforms_pairs,growth_ratio
1,0-5x-long-algorand-token,0.5X Long Algorand Token,algohalf,NaN,NaN,100.00,0.00,NaN,5964.0,0.000,...,0,0,0,0,0,0,5253.0,NaN,[],1.780830
2,0-5x-long-altcoin-index-token,0.5X Long Altcoin Index Token,althalf,NaN,NaN,100.00,0.00,NaN,6118.0,0.000,...,0,0,0,0,0,0,5253.0,NaN,"[{'platorm': 'FTX', 'pair': 'ALTHALF/USD'}]",2.052589
3,0-5x-long-balancer-token,0.5X Long Balancer Token,balhalf,NaN,NaN,NaN,NaN,NaN,6083.0,0.000,...,0,0,0,0,0,0,5253.0,NaN,[],1.459818
4,0-5x-long-bitcoin-cash-token,0.5X Long Bitcoin Cash Token,bchhalf,NaN,NaN,NaN,NaN,NaN,6026.0,0.000,...,0,0,0,0,0,0,5253.0,NaN,[],1.400993
5,0-5x-long-bitcoin-sv-token,0.5X Long Bitcoin SV Token,bsvhalf,NaN,NaN,NaN,NaN,NaN,5746.0,0.200,...,0,0,0,0,0,0,5253.0,NaN,"[{'platorm': 'FTX', 'pair': 'BSVHALF/USD'}]",1.286724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6241,zuplo,Zuplo,zlp,PA,NaN,NaN,NaN,NaN,5842.0,0.200,...,0,0,0,0,0,0,0.0,NaN,"[{'platorm': 'ProBit', 'pair': 'ZLP/ETH'}, {'p...",580.989903
6243,zyro,Zyro,zyro,NaN,NaN,100.00,0.00,NaN,3601.0,3.812,...,0,0,0,0,0,0,1001073.0,NaN,"[{'platorm': 'OKEx', 'pair': 'ZYRO/USDT'}, {'p...",38.179615
6244,zyx,ZYX,zyx,ES,NaN,100.00,0.00,NaN,4078.0,2.366,...,0,0,0,0,0,0,473318.0,NaN,"[{'platorm': 'Bitforex', 'pair': 'ZYX/USDT'}, ...",327.292546
6245,zzz-finance,zzz.finance,zzz,US,NaN,0.00,100.00,2205.0,2585.0,8.931,...,0,0,0,0,0,0,70612.0,NaN,"[{'platorm': '1inch', 'pair': '0XC75F15ADA5812...",5238.831676


In [75]:
def contains_binance(list_):
    for platform_pair in eval(list_):
        if platform_pair['platorm'].lower() == 'binance':
            return True
    return False
            
coins_binance_df = coins_df[coins_df['platforms_pairs'].apply(contains_binance)]

In [76]:
coins_binance_df

,id,name,symbol,country_origin,genesis_date,sentiment_votes_up_percentage,sentiment_votes_down_percentage,market_cap_rank,coingecko_rank,coingecko_score,...,git_stars,git_subscribers,git_total_issues,git_closed_issues,git_pull_requests_merged,git_pull_request_contributors,alexa_rank,bing_matches,platforms_pairs,growth_ratio
33,0x,0x,zrx,US,2017-08-11,89.01,10.99,53.0,26.0,56.558,...,1305,82,447,432,1800,76,88488.0,NaN,"[{'platorm': 'ZBG', 'pair': 'ZRX/USDT'}, {'pla...",20.718175
42,1inch,1inch,1inch,NaN,NaN,94.85,5.15,91.0,455.0,28.190,...,0,0,0,0,0,0,NaN,NaN,"[{'platorm': 'Binance', 'pair': '1INCH/USDT'},...",8.179373
193,aave,Aave,aave,NaN,NaN,78.81,21.19,15.0,48.0,51.824,...,485,53,19,14,4,2,15890.0,NaN,"[{'platorm': 'Binance', 'pair': 'AAVE/USDT'}, ...",22.104151
199,aavegotchi,Aavegotchi,ghst,VG,NaN,92.59,7.41,479.0,1061.0,20.008,...,0,0,0,0,0,0,179954.0,NaN,"[{'platorm': 'Binance', 'pair': 'GHST/BUSD'}, ...",1.991978
241,adex,AdEx,adx,RU,2017-07-01,100.00,0.00,273.0,133.0,42.027,...,24,5,20,14,25,7,13684.0,NaN,"[{'platorm': 'Binance', 'pair': 'ADX/BTC'}, {'...",99.387613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6125,yoyow,YOYOW,yoyow,CN,2017-08-26,50.00,50.00,978.0,1361.0,17.453,...,0,0,0,0,0,0,3015981.0,NaN,"[{'platorm': 'Binance', 'pair': 'YOYO/BTC'}, {...",124.922214
6154,zcash,Zcash,zec,NaN,2016-10-28,90.24,9.76,49.0,15.0,61.405,...,4319,395,3089,2220,1080,81,167113.0,NaN,"[{'platorm': 'Bitfinex', 'pair': 'ZEC/USD'}, {...",161.616709
6158,zcoin,Firo,firo,NaN,2016-09-28,86.11,13.89,290.0,530.0,26.686,...,0,0,0,0,0,0,286121.0,NaN,"[{'platorm': 'Binance', 'pair': 'FIRO/BTC'}, {...",499.908795
6174,zencash,Horizen,zen,NaN,NaN,91.30,8.70,93.0,61.0,49.803,...,99,33,88,76,243,28,249437.0,NaN,"[{'platorm': 'ZBG', 'pair': 'ZEN/USDT'}, {'pla...",19.981595


In [77]:
coins_binance_df[coins_binance_df['growth_ratio']<=2]

,id,name,symbol,country_origin,genesis_date,sentiment_votes_up_percentage,sentiment_votes_down_percentage,market_cap_rank,coingecko_rank,coingecko_score,...,git_stars,git_subscribers,git_total_issues,git_closed_issues,git_pull_requests_merged,git_pull_request_contributors,alexa_rank,bing_matches,platforms_pairs,growth_ratio
199,aavegotchi,Aavegotchi,ghst,VG,NaN,92.59,7.41,479.0,1061.0,20.008,...,0,0,0,0,0,0,179954.0,NaN,"[{'platorm': 'Binance', 'pair': 'GHST/BUSD'}, ...",1.991978
721,binance-eth,Binance ETH,beth,NaN,NaN,80.00,20.00,NaN,2884.0,6.967,...,0,0,0,0,0,0,NaN,NaN,"[{'platorm': 'Binance', 'pair': 'BETH/ETH'}, {...",1.473392
722,binanceidr,BIDR,bidr,NaN,NaN,75.00,25.00,NaN,2586.0,8.925,...,0,0,0,0,0,0,211700.0,NaN,"[{'platorm': 'Binance', 'pair': 'BTC/BIDR'}, {...",1.132720
723,binance-krw,Binance KRW,BKRW,NaN,NaN,NaN,NaN,NaN,3554.0,3.989,...,0,0,0,0,0,0,0.0,NaN,"[{'platorm': 'Binance', 'pair': 'USDT/BKRW'}, ...",1.648485
724,binance-usd,Binance USD,busd,NaN,NaN,70.00,30.00,46.0,278.0,33.570,...,0,0,0,0,0,0,82114.0,NaN,"[{'platorm': 'Binance', 'pair': 'BUSD/USDT'}, ...",1.268295
1614,dai,Dai,dai,NaN,NaN,71.43,28.57,41.0,174.0,38.498,...,0,0,0,0,0,0,30241.0,NaN,"[{'platorm': 'Sushiswap', 'pair': '0X6B175474E...",1.350689
4132,pax-gold,PAX Gold,paxg,NaN,NaN,100.00,0.00,197.0,786.0,22.907,...,0,0,0,0,0,0,82114.0,NaN,"[{'platorm': 'Binance', 'pair': 'PAXG/USDT'}, ...",1.539760
4133,paxos-standard,Paxos Standard,pax,NaN,NaN,66.67,33.33,80.0,131.0,42.134,...,58,22,5,5,9,3,82114.0,NaN,"[{'platorm': 'Bitstamp', 'pair': 'BTC/PAX'}, {...",1.244254
5468,true-usd,TrueUSD,tusd,NaN,2018-03-05,50.00,50.00,108.0,86.0,46.656,...,178,24,26,24,379,16,223259.0,NaN,"[{'platorm': 'Binance', 'pair': 'TUSD/USDT'}, ...",1.833513
5633,usd-coin,USD Coin,usdc,US,NaN,70.00,30.00,13.0,72.0,48.379,...,167,18,28,25,251,16,65370.0,NaN,"[{'platorm': 'Serum DEX', 'pair': 'BTC/USDC'},...",1.265976


In [28]:
coins_df_date[coins_df_date['genesis_date']>= a_year_ago][['name', 'ath', 'atl', 'growth_ratio']]


,name,ath,atl,growth_ratio
181,5G-CASH,0.123068,5.651000e-05,2.177809e+03
602,BARE,3.400000,1.033016e-02,3.291333e+02
816,BitcoinStaking,0.510395,1.579800e-04,3.230757e+03
1590,CURVEHASH,0.403112,9.990000e-06,4.035155e+04
2076,Enecuum,0.206364,4.868100e-03,4.239108e+01
2648,GrafenoCoin,0.001700,2.000000e-07,8.502000e+03
2931,Improved Bitcoin,0.110735,1.586000e-05,6.982030e+03
3438,MahaDAO,25.150000,1.980000e+00,1.270202e+01
3510,MCDEX,15.750000,9.114020e-01,1.728107e+01
3880,Nibble,0.066080,6.257000e-05,1.056097e+03


In [42]:
coins_df_date[(coins_df_date['price_change_percentage_200d']<-70) & (coins_df_date['market_cap']<1000000)]

,Unnamed: 0,id,name,symbol,country_origin,genesis_date,sentiment_votes_up_percentage,sentiment_votes_down_percentage,market_cap_rank,coingecko_rank,...,telegram_channel_user_count,git_stars,git_subscribers,git_total_issues,git_closed_issues,git_pull_requests_merged,git_pull_request_contributors,alexa_rank,bing_matches,growth_ratio
717,717,billionaire-token,Billionaire Token,xbl,NaN,2017-08-01,NaN,NaN,2092.0,1530.0,...,NaN,3,1,0,0,0,0,0.0,NaN,6.056093e+03
969,969,blockmason-credit-protocol,Blockmason Credit Protocol,bcpt,NaN,2017-10-18,50.00,50.00,1453.0,194.0,...,NaN,27,14,14,14,22,5,2220119.0,NaN,4.160013e+02
1063,1063,boxx,Blockparty,boxx,US,2018-05-29,NaN,NaN,2104.0,2290.0,...,205.0,0,0,0,0,0,0,4318336.0,NaN,1.155810e+04
1116,1116,buggyra-coin-zero,Buggyra Coin Zero,bczero,NaN,2018-06-05,NaN,NaN,NaN,5260.0,...,61.0,0,0,0,0,0,0,0.0,NaN,1.769897e+03
1204,1204,casinocoin,Casinocoin,csc,NaN,2013-07-19,85.71,14.29,NaN,1643.0,...,NaN,0,0,0,0,12,1,776691.0,NaN,1.032295e+04
1370,1370,coinus,CoinUs,cnus,KR,2018-11-13,NaN,NaN,1860.0,2413.0,...,705.0,0,0,0,0,0,0,8161012.0,NaN,1.418479e+04
1406,1406,condensate,Condensate,rain,NaN,2017-04-29,NaN,NaN,NaN,1937.0,...,NaN,9,6,4,1,7,4,8895102.0,NaN,1.248845e+05
1441,1441,counos-coin,Counos Coin,cca,CH,2018-03-01,NaN,NaN,NaN,3086.0,...,669.0,3,0,0,0,0,0,886294.0,NaN,2.416726e+05
1561,1561,cryptoworld-vip,CryptoWorld.VIP,cwv,NaN,2018-05-31,NaN,NaN,NaN,2939.0,...,85.0,70,0,0,0,0,0,3117835.0,NaN,7.448092e+02
1688,1688,decentbet,DecentBet,dbet,NaN,2017-09-23,NaN,NaN,2139.0,693.0,...,NaN,0,0,0,0,0,0,1128238.0,NaN,2.208612e+03
